In [1]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import json
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [1]:
vocab_size = 10000
embedding_dim = 16
max_length = 30
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 136054
file_path = os.path.join(os.path.dirname(os.getcwd()),"data\\cleaned_data_en_4file.json")
model_save_path = os.path.join(os.path.dirname(os.getcwd()),"models\\twitter_LSTM_model")

In [4]:
with open(file_path, 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['text'])
    labels.append(item['viral'])

In [5]:
print("sentences: ", len(sentences))
print("labels", labels[1:10])

sentences:  170068
labels [0, 0, 1, 0, 0, 0, 0, 0, 0]


In [6]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [7]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [8]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.GlobalAveragePooling1D(),
    # tf.keras.layers.Dense(24, activation='relu'),
    # tf.keras.layers.Dense(1, activation='sigmoid')
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
sgdm_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=sgdm_optimizer, metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 16)            160000    
                                                                 
 bidirectional (Bidirectiona  (None, 30, 128)          41472     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 246,913
Trainable params: 246,913
Non-trai

In [11]:
num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/50
4252/4252 - 64s - loss: 0.5110 - accuracy: 0.7732 - val_loss: 0.1125 - val_accuracy: 0.9867 - 64s/epoch - 15ms/step
Epoch 2/50
4252/4252 - 58s - loss: 0.1439 - accuracy: 0.9560 - val_loss: 0.0635 - val_accuracy: 0.9904 - 58s/epoch - 14ms/step
Epoch 3/50
4252/4252 - 58s - loss: 0.0886 - accuracy: 0.9789 - val_loss: 0.0619 - val_accuracy: 0.9904 - 58s/epoch - 14ms/step
Epoch 4/50
4252/4252 - 58s - loss: 0.0820 - accuracy: 0.9799 - val_loss: 0.0550 - val_accuracy: 0.9918 - 58s/epoch - 14ms/step
Epoch 5/50
4252/4252 - 58s - loss: 0.0728 - accuracy: 0.9818 - val_loss: 0.0381 - val_accuracy: 0.9955 - 58s/epoch - 14ms/step
Epoch 6/50
4252/4252 - 58s - loss: 0.0682 - accuracy: 0.9827 - val_loss: 0.0380 - val_accuracy: 0.9955 - 58s/epoch - 14ms/step
Epoch 7/50
4252/4252 - 58s - loss: 0.0594 - accuracy: 0.9844 - val_loss: 0.0301 - val_accuracy: 0.9972 - 58s/epoch - 14ms/step
Epoch 8/50
4252/4252 - 58s - loss: 0.0468 - accuracy: 0.9870 - val_loss: 0.0246 - val_accuracy: 0.9981 - 58s/ep

In [12]:
model.save("twitter_LSTM_sgdm_model_001.h5")

In [14]:
sentence = ["btw if the virus cases dont explode exponentially in a few days,  imma be real suspicious !"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[0.99978054]]
